## Introduction
This notebook runs all the processing steps one by one for several models and renders the output. Each section is individually runnable after a kernel restart 

## Observations
* Symbolic tracing did not play well with any BERT model, because it creates proxies for mutually exclusive inputs to e.g. `DistilBertModel.forward`
  * This was fixed by making the `concrete_args` input to `fx.symbolic_trace` available to the `MAV` and `MavTracer` objects
  * For BERT models, `concrete_args={'inputs_embeds':None}` gets around this issue
* Still, most NLP models use proxy variables for control flow, which is not supported by `torch.fx`
  * Perhaps fixing more arguments via `concrete_args` could work around this. To be investigated.

## DistilBERT

In [ ]:
import sys
sys.path.append('..')
from transformers import DistilBertModel, DistilBertTokenizer
import torch
from idlmav import MAV, plotly_renderer

model = DistilBertModel.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model.eval()
inputs = tokenizer("Hello world", return_tensors="pt")
device = 'cpu'

mav = MAV(model, inputs, concrete_args={'inputs_embeds':None})
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


INFO:2025-02-13 19:34:11 23465:23465 init.cpp:181] If you see CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, refer to https://developer.nvidia.com/nvidia-development-tools-solutions-err-nvgpuctrperm-cupti



## T5-small encoder

In [2]:
import sys
sys.path.append('..')
from transformers import T5Model, T5Tokenizer
import torch
from idlmav import MAV, plotly_renderer

model = T5Model.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model.eval()
inputs = tokenizer("translate English to French: Hello, how are you?", return_tensors="pt")
device = 'cpu'

mav = MAV(model.encoder, inputs, device=device, concrete_args={'inputs_embeds':None})
with plotly_renderer('notebook_connected'): mav.show_figure()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Tracing failed with torch.fx.symbolic_trace: finfo(): argument 'type' (position 1) must be torch.dtype, not Attribute
Tracing with torch.compile


/home/dev/ai/idlmav/nbs/../idlmav/tracing.py:217: UserWarning:

Forward pass failed. Last successful node: "relative_buckets_1:iadd". Possible error node: "values:embedding".: index out of range in self

While executing %values : [num_users=1] = call_function[target=torch.nn.functional.embedding](args = (%relative_buckets_1, %l_self_modules_block_modules_0_modules_layer_modules_0_modules_self_attention_modules_relative_attention_bias_parameters_weight_, None, None, 2.0, False, False), kwargs = {})
Original traceback:
  File "/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/transformers/models/t5/modeling_t5.py", line 1124, in forward
    layer_outputs = layer_module(
  File "/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/transformers/models/t5/modeling_t5.py", line 675, in forward
    self_attention_outputs = self.layer[0](
  File "/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/transformers/models/t5/modeling_t5.py", line 593, in forward
    attention_output = sel

## BERT mini

In [3]:
import sys
sys.path.append('..')
from transformers import BertModel, BertTokenizer
import torch
from idlmav import MAV, plotly_renderer

model = BertModel.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
tokenizer = BertTokenizer.from_pretrained("google/bert_uncased_L-2_H-128_A-2")
model.eval()
inputs = tokenizer("This is a test sentence.", return_tensors="pt")
device = 'cpu'

mav = MAV(model, inputs, concrete_args={'inputs_embeds':None})
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


In [4]:
mav.show_widget(add_overview=True)

## ALBERT Lite

In [5]:
import sys
sys.path.append('..')
from transformers import AlbertModel, AlbertTokenizer
import torch
from idlmav import MAV, plotly_renderer

model = AlbertModel.from_pretrained("albert-base-v2")
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
model.eval()
inputs = tokenizer("The quick brown fox jumps over the lazy dog.", return_tensors="pt")
device = 'cpu'

mav = MAV(model, inputs, concrete_args={'inputs_embeds':None})
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


## ModernBERT

In [1]:
import sys
sys.path.append('..')
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from idlmav import MAV, plotly_renderer, MavTracer

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)
inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
device='cpu'

mav = MAV(model, inputs, concrete_args={'inputs_embeds':None})
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


INFO:2025-02-13 11:43:53 20271:20271 init.cpp:181] If you see CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, refer to https://developer.nvidia.com/nvidia-development-tools-solutions-err-nvgpuctrperm-cupti


Total nodes: 1349. Input nodes: 92. Output nodes: 67


/home/dev/ai/idlmav/nbs/../idlmav/layout.py:72: UserWarning:

The largest level has 92 nodes. This is an indication that something may have gone wrong during the tracing step

/home/dev/ai/idlmav/nbs/../idlmav/layout.py:73: UserWarning:

Falling back to greedy layout algorithm



In [ ]:
tracer = MavTracer(model, inputs, device, concrete_args={'inputs_embeds':None})

In [2]:
[n.name for n in tracer.g.in_nodes]

['l_input_ids_',
 'l_attention_mask_',
 'arange',
 'arange_1',
 '_set_grad_enabled',
 'getitem_1',
 '_enter_autocast',
 '_set_grad_enabled_1',
 '_set_grad_enabled_2',
 'getitem_12',
 '_enter_autocast_1',
 '_set_grad_enabled_3',
 '_set_grad_enabled_4',
 'getitem_23',
 '_enter_autocast_2',
 '_set_grad_enabled_5',
 '_set_grad_enabled_6',
 'getitem_34',
 '_enter_autocast_3',
 '_set_grad_enabled_7',
 '_set_grad_enabled_8',
 'getitem_45',
 '_enter_autocast_4',
 '_set_grad_enabled_9',
 '_set_grad_enabled_10',
 'getitem_56',
 '_enter_autocast_5',
 '_set_grad_enabled_11',
 '_set_grad_enabled_12',
 'getitem_67',
 '_enter_autocast_6',
 '_set_grad_enabled_13',
 '_set_grad_enabled_14',
 'getitem_78',
 '_enter_autocast_7',
 '_set_grad_enabled_15',
 '_set_grad_enabled_16',
 'getitem_89',
 '_enter_autocast_8',
 '_set_grad_enabled_17',
 '_set_grad_enabled_18',
 'getitem_100',
 '_enter_autocast_9',
 '_set_grad_enabled_19',
 '_set_grad_enabled_20',
 'getitem_111',
 '_enter_autocast_10',
 '_set_grad_enabl

In [2]:
len(tracer.g.nodes)

1349